Logistic Regression

In [30]:
import numpy as np
from tqdm import tqdm
from tensorflow import keras
import tensorflow

### Importing HuggingFace

In [2]:
!pip install datasets
!pip install contractions
!pip install beautifulsoup4

In [1]:
from datasets import load_dataset

In [3]:
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

Reusing dataset imdb (/home/token/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/home/token/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [4]:
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [5]:
print(type(train_dataset['text']))

<class 'list'>


In [6]:
print(train_dataset[0])

{'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!', 'label': 1}


In [109]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import numpy as np
import re

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text):
    return contractions.fix(text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

def pre_process_document(document):
    # strip HTML
    document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    document = remove_accented_chars(document)
    # expand contractions    
    document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    #special_char_pattern = re.compile(r'([{.(-)}])')
    #document = special_char_pattern.sub(" \\1 ", document)
    #document = remove_special_characters(document, remove_digits=True)  
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

In [111]:
x_train = pre_process_corpus(train_dataset['text'])
x_train_preprocessed = []

for elt in x_train:
    x_train_preprocessed.append(np.array(elt.split(" ")))
    
y_train = train_dataset['label']



x_test_preprocessed = []

x_test = pre_process_corpus(test_dataset['text'])
y_test = test_dataset['label']

for elt in x_test:
    x_test_preprocessed.append(np.array(elt.split(" ")))

In [112]:
print(x_train_preprocessed[0])
print(x_test_preprocessed[0])

['bromwell' 'high' 'is' 'a' 'cartoon' 'comedy.' 'it' 'ran' 'at' 'the'
 'same' 'time' 'as' 'some' 'other' 'programs' 'about' 'school' 'life,'
 'such' 'as' '"teachers".' 'my' '35' 'years' 'in' 'the' 'teaching'
 'profession' 'lead' 'me' 'to' 'believe' 'that' 'bromwell' "high's"
 'satire' 'is' 'much' 'closer' 'to' 'reality' 'than' 'is' '"teachers".'
 'the' 'scramble' 'to' 'survive' 'financially,' 'the' 'insightful'
 'students' 'who' 'can' 'see' 'right' 'through' 'their' 'pathetic'
 "teachers'" 'pomp,' 'the' 'pettiness' 'of' 'the' 'whole' 'situation,'
 'all' 'remind' 'me' 'of' 'the' 'schools' 'i' 'knew' 'and' 'their'
 'students.' 'when' 'i' 'saw' 'the' 'episode' 'in' 'which' 'a' 'student'
 'repeatedly' 'tried' 'to' 'burn' 'down' 'the' 'school,' 'i' 'immediately'
 'recalled' '.........' 'at' '..........' 'high.' 'a' 'classic' 'line:'
 'inspector:' 'I' 'am' 'here' 'to' 'sack' 'one' 'of' 'your' 'teachers.'
 'student:' 'welcome' 'to' 'bromwell' 'high.' 'i' 'expect' 'that' 'many'
 'adults' 'of' 

### Logistic Regression

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [10]:
def import_lexicon(path):
    data = pd.read_csv(path, sep='\t', names=[0, 1, 2, 3])
    df = pd.DataFrame()
    df['token'] = data[0]
    df['sentiment'] = data[1]
    return df

In [17]:
def does_no_appear(review) -> int:
    if "no" in review:
        return 1
    return 0

In [18]:
def count_first_and_second_pro(review) -> int:
    count = 0
    for word in review:
        if word in ["I", "i", "you", "yours"]:
            count += 1
    return count

Let's search for all the words finishing with !

As it would make no sense that the ! character appears before the end of the word

In [19]:
def does_exclamation_appear(review) -> int:
    if "!" in review:
        return 1
    return 0

it appears no words ends with ! in the training set

Thus, there is no need to check for ! in the documents

In [32]:
def log_word_count_in_doc(review):
    return np.log(len(review))

In [ ]:
def filter_lexicon(lexicon, word_index):
    # only the int value of the words present in index
    filtered_lexicon, i = pd.DataFrame([], columns=['token_int', 'token_string', 'sentiment']), 0
    for index in range(lexicon.shape[0]):
        token = lexicon['token'][index]
        if token in word_index.keys():
            mapped_token = word_index[token]
            filtered_lexicon.loc[i] = [mapped_token, lexicon.token[index], lexicon.sentiment[index]]
            i += 1
    return filtered_lexicon

In [26]:
def split_lexicon(lexicon):
    positive_df = lexicon[lexicon.sentiment > 0]
    positive_words = positive_df['token'].to_numpy().tolist()
    negative_df = lexicon[lexicon.sentiment < 0]
    negative_words = negative_df['token'].to_numpy().tolist()
    return positive_words, negative_words

In [116]:
def number_of_words_pos(review, positive_words):
    posi = np.isin(positive_words, review)
    return sum(posi)

def number_of_words_neg(review, negative_words):
    nega = np.isin(review, negative_words)
    return sum(nega)

In [117]:
def LoRegression(x_train, y_train):
    lexicon = import_lexicon("vader_lexicon.txt")
    positive_words, negative_words = split_lexicon(lexicon)
    X_features = []
    for review in tqdm(x_train):
        feature = np.zeros(6)
        feature[0] = does_no_appear(review)
        feature[1] = does_exclamation_appear(review)
        feature[2] = count_first_and_second_pro(review)
        feature[3] = log_word_count_in_doc(review)
        feature[4] = number_of_words_neg(review, negative_words)
        feature[5] = number_of_words_pos(review, positive_words)

        X_features.append(feature)
    return np.asarray(X_features)

In [118]:
X_train_features = LoRegression(x_train_preprocessed, y_train)

100%|██████████| 25000/25000 [13:37<00:00, 30.57it/s]


In [119]:
X_train_features.shape

(25000, 6)

In [120]:
clf = LogisticRegression(random_state=0).fit(X_train_features, y_train)

In [121]:
#create X_test using loRegression to have a usable informations
X_test_features = LoRegression(x_test_preprocessed, y_test)

100%|██████████| 25000/25000 [14:49<00:00, 28.09it/s]


In [122]:
y_pred = clf.predict(X_test_features)
print(X_test_features.shape, y_pred.shape)

(25000, 6) (25000,)


In [123]:
clf.score(X_test_features, y_test)

0.69612

In [124]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.70      0.70      0.70     12500
     class 1       0.70      0.70      0.70     12500

    accuracy                           0.70     25000
   macro avg       0.70      0.70      0.70     25000
weighted avg       0.70      0.70      0.70     25000

